# Map mRNA Counts by Regions and Taxa

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'mRNA_Maps'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.patches as patches
import seaborn as sns
import zipfile
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Make some choices...
dataset = 'Y23' # Y21ss','Y21tenX' or 'Y23'
geneset = '30NPPs'
#geneset = '30NPPs*Rs'
#geneset = '61NPPs'
#geneset = '77NPRs'

#type_id = 'All' # 'All', '01 Neuronal', '02 Non-Neuronal' (Y21), '02 Glial' (Y23)
type_id = '01 Neuronal' 

# Assemble the general filepaths
name_root = notebook_name
source_root = 'NP'
#source_root = 'nMod'
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

# Read annotated expression data
source_path = 'Resources/'+dataset+'_'+source_root+'_counts.csv.zip' # 'NP' or 'nMod'
support_path = 'Resources/'+dataset+'_'+source_root+'_support.json' # 'NP' or 'nMod'

print('Reading',source_path)
df = pd.read_csv(source_path)

# Filter based on type ID
if type_id != 'All': anno_counts = df[df['type'] == type_id].copy()
else: anno_counts = df.copy()

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_classes = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_classes = {key: subclass_classes[key] for key in sorted(subclass_classes)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')
region_names = df.set_index('region_label')['region_name'].to_dict()
region_shorts = df.set_index('region_label')['region_short'].to_dict()
#region_zones = df.set_index('region_short')['zone'].to_dict()
region_zones = df.set_index('region_label')['zone'].to_dict()
zone_regions = {}
for key, value in region_zones.items():
    if value not in zone_regions:
        zone_regions[value] = [key]
    else:
        zone_regions[value].append(key)

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())

# Read a table to form gene label dictionary
try: df = pd.read_csv('Resources/'+geneset+'.csv',usecols=[0,1], index_col=0)
except: df = pd.read_csv('Resources/All_nMods_labeled.csv', usecols=[1,2], index_col=0)
gene_labels = df.to_dict()['Label']

# Configure multiindex
major_annos = ['type','region','class','subclass','cluster']
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos, inplace=True)
anno_counts.set_index(major_annos, inplace=True)

# Read in a genelist and corresponding counts data
convert_to_aggrs = False if geneset[-3:] != '*Rs' else True
if convert_to_aggrs: geneset = geneset[0:-3]

df = pd.read_csv('Resources/'+geneset+'.csv')
genelist = list(df['Symbol'])   
counts = anno_counts[genelist].copy()


if convert_to_aggrs:
    # Convert an NPP matrix to an aggregated, conjugate NP-GPCR matrix
    assert 'NPP' in geneset, 'This cell requires an NPP gene list!'

    newlist_name = geneset+'*Rs'
    print('Converting', geneset, 'to', newlist_name)
    # Use conjugate pair table to generate aggregate NP_GPCRs conjugate to a set of NPPs
    # Read pair table
    pair_table = pd.read_csv('Resources/Mouse_NP_pairs.csv')
    pair_table['Pair Label'] = pair_table['NPP'] + '->' + pair_table['NP-GPCR']
    pair_id = list(pair_table['Pair ID'])
    pair_npps = list(pair_table['NPP'])
    pair_nprs = list(pair_table['NP-GPCR'])
    pair_labels = list(pair_table['Pair Label'])
    pair_table.set_index('Pair ID', inplace=True)
    # Build cognate NPP/aggregate-NP-GPCRs dictionary
    df = pair_table[['NPP','NP-GPCR']].set_index('NPP').copy()
    df['Aggs'] = df['NP-GPCR']+' '
    df = df.groupby('NPP').sum()
    aggs_dict = df['Aggs'].to_dict()
    for npp in aggs_dict: aggs_dict[npp] = aggs_dict[npp].split()
    # Build dataframe with aggregated NP-GPCRs conjugate to included NPPs
    npp_list = genelist
    df = anno_counts[npp_list].copy()
    for npp in npp_list: df[npp+'Rs'] = anno_counts[aggs_dict[npp]].sum(axis=1)
    npr_list = [x+'Rs' for x in npp_list]
    counts = df.copy()

    geneset = newlist_name
    genelist = npr_list
    counts = counts[npr_list].copy()

print('Genelist:',geneset,', Counts matrix shape:', counts.shape)
gene_order = genelist

In [ ]:
# Make some more general choices
row_norm = True
log_floor_coef=0.01
sort = False
#sort = True
#yticklabels=['('+gene_labels[s]+') '+s for s in genelist]
yticklabels=genelist
slant = 90
draw_vlines = True

# Generate filename root
root = dataset
if type_id == 'All': root += '_all_'
elif type_id == '01 Neuronal': root += '_neur_'
else: root += type_id+'_'
root += geneset+'_'
root


In [ ]:
# Map regions, write results to a PDF and a ZIP archive of CSVs

font_scale = 1.1
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})

metric = 'cell%' # 'sum','mean', 'cell%', 'clus%'
row_stat='max' # 'max', 'mean', 'sum'

# matter only for clus% metric
cluster_thresh_factor = 0.5
min_cluster = 10

min_region = 10

log = False if '%' in metric else True
threshold = 10 if 'ss' in dataset else 1
if metric == 'clus%': threshold *= cluster_thresh_factor * threshold

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle

handle = root+'regions_'+metric+'_'+row_stat+'_'
handle += 'minReg' + str(min_region) + '_'
#handle += 'minSubC' + str(min_subclass) + '_'
if 'clus' in metric: handle += 'minClus'+str(min_cluster)+'_'
if '%' in metric: handle += 'Thr'+str(threshold)+'_'

pdf_file = pdf_path+handle+sn+'.pdf'
zip_file = zip_path+handle+sn+'.zip'
temp_csv = temp_path+sn+'temp.csv'
pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

#for cell_class in ['00 All']: # for quick short result
#for cell_class in ['01 Glutamatergic']: # for quick short result
for cell_class in ['00 All']+classes:
    df = counts.loc[idx[types,regions,classes,subclasses], :]
    if cell_class != '00 All': df = df.xs(cell_class, level='class')
    df = df.groupby('region').filter(lambda x: len(x) >= min_region) # filter out small groups
    if len(df) == 0: continue
        
    if metric == 'sum': # Sum values
        X = df.groupby('region').sum().T

    if metric == 'mean': # Find mean values
        X = df.groupby('region').mean().T
    
    if metric == 'cell%': # Find percent expressing by cell
        X = 100*(df+1-threshold).clip(0,1).groupby('region').mean().T

    if metric == 'clus%': # Find percent expressing by cluster
        pre = len(df)
        df = df.groupby('cluster').filter(lambda x: len(x) >= min_cluster) # filter out small clusters
        post = len(df)
        print(pre - post, 'cells filtered out of', pre, 'due due to cluster size <', min_cluster)
        cluster_mean = df.groupby(['region','cluster']).mean()
        cluster_hot = (cluster_mean >= threshold).astype(int)
        cluster_frac = cluster_hot.groupby('region').mean()
        X = 100 * cluster_frac.T

    # Make two useful lists
    column_labels = [region_shorts[s] for s in list(X.columns)]
    #column_labels = [region_names[s] for s in list(X.columns)]
    column_colors = [region_colors[s] for s in list(X.columns)]

    # Get unit tallies
    cell_tally = df.groupby('region').count().iloc[:,0].T
    cell_tally.index = column_labels
        
    # Find zone boundaries if desired
    if draw_vlines:
        vlines = []; zone = []
        #for i, region in enumerate([region_shorts[s] for s in list(X.columns)]):
        for i, region in enumerate(X.columns):
            if region_zones[region] != zone: vlines += [i]; zone = region_zones[region]
        vlines += [len(X.columns)]        
    else: vlines = None
        
    # Draw subclass heatmap
    title = handle+sn+'\nClass: '+cell_class+' ('+metric+')'
    if '%' in metric: cbar_label = '%Counts>='+str(threshold); units =' %'
    else: cbar_label = 'Mean_Counts'; units = ''

    X.index = yticklabels
    X.columns = column_labels
    row_stats = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log,
                                row_stat=row_stat, log_floor_coef=log_floor_coef, vlines = vlines,
                                row_sort=sort, slant=slant, cbar=True, cbar_label=cbar_label, units=units)
    
    # Write row statistics to CSV file
    row_stats.index.name = cell_class.split()[1]
    row_stats.to_csv(csv_path+handle+'Cla'+cell_class.split()[0]+'_'+sn+'.csv')
    
    # Append cell tally row and add to Zip file 
    X.loc['cells'] = cell_tally
    X.to_csv(temp_csv)
    with zipfile.ZipFile(zip_file, zip_mode) as zipf:
        arc_name = handle+cell_class.split()[0]+'.csv'
        zipf.write(temp_csv, arcname=arc_name)
    zip_mode = 'a'    
    print(arc_name,'added to:',zip_file, '\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)


In [ ]:
# Map subclasses by region, write results to a PDF and a ZIP archive of CSVs

font_scale = 1.1
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})

metric = 'cell%' # 'sum','mean', 'cell%', 'clus%'
row_stat='max' # 'max', 'mean', 'sum'

cluster_thresh_factor = 0.5
min_cluster = 10

# Set minimum subclass size according to dataset choice
min_subclass = 10 if 'ss' in dataset else 100
#min_subclass = 10 if 'ss' in dataset else 1

log = False if '%' in metric else True
threshold = 10 if 'ss' in dataset else 1
if metric == 'clus%': threshold *= cluster_thresh_factor * threshold
    
#threshold = 100 # Ad Hoc...

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root + 'subsByReg_' + metric + '_'
handle += 'minSubC' + str(min_subclass) + '_'
if 'clus' in metric: handle += 'minClus'+str(min_cluster)+'_'
if '%' in metric: handle += 'Thr'+str(threshold)+'_'
handle += sn
print(handle)

pdf_file = pdf_path+handle+'.pdf'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'

pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

# for quick, short results
#for region in ['Neocortex']:
#for region in ['003 SSp']:
#for region in common_regions:
#for region in ['001 VISp','007 SSp','012 ALM','028 HIP']:
for region in ['018 STRd','019 STRv']:
#for region in regions:
    if region[0] == '0': # Leading zero indicates one specific region
        df = counts.loc[idx[types,region,classes,subclasses], :]
        region_label = region_shorts[region]
    else: # Otherwise, interpret as a list of regions to aggregate
        df = counts.loc[idx[types,zone_regions[region],classes,subclasses], :]
        region_label = region
    pre = len(df)
    df = df.groupby('subclass').filter(lambda x: len(x) >= min_subclass) # filter out small subclasses
    post = len(df)
    print(pre - post, 'cells filtered out of', pre, 'due to subclass size <', min_subclass)
    if len(df) == 0: continue
        
    if metric == 'sum': # Sum values
        X = df.groupby('subclass').sum().T

    if metric == 'mean': # Find mean values
        X = df.groupby('subclass').mean().T
    
    if metric == 'cell%': # Find percent expressing by cell
        X = 100 * (df >= threshold).astype(int).groupby('subclass').mean().T

    if metric == 'clus%': # Find percent expressing by cluster
        pre = len(df)
        df = df.groupby('cluster').filter(lambda x: len(x) >= min_cluster) # filter out small clusters
        post = len(df)
        print(pre - post, 'cells filtered out of', pre, 'due due to cluster size <', min_cluster)
        cluster_mean = df.groupby(['subclass','cluster']).mean()
        cluster_hot = (cluster_mean >= threshold).astype(int)
        cluster_frac = cluster_hot.groupby('subclass').mean()
        X = 100 * cluster_frac.T

    # Make two useful lists
    column_labels = [subclass_shorts[s] for s in list(X.columns)]
    column_colors = [subclass_colors[s] for s in list(X.columns)]

    # Get unit tallies
    cell_tally = df.groupby('subclass').count().iloc[:,0].T
    cell_tally.index = column_labels
        
    # Find zone boundaries if desired
    if draw_vlines:
        vlines = []; zone = []
        for i, subclass in enumerate(X.columns):
            if subclass_classes[subclass] != zone: vlines += [i]; zone = subclass_classes[subclass]
        vlines += [len(X.columns)]
    else: vlines = None

    # Draw subclass heatmap
    title = handle+'\n'+sn+' Region: '+region_label+' ('+metric+')'
    if '%' in metric: cbar_label = '%Counts>='+str(threshold); units =' %'
    else: cbar_label = 'Mean_Counts'; units = ''

    X.index = yticklabels
    X.columns = column_labels
    row_stats = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log,
                                row_stat=row_stat, log_floor_coef=log_floor_coef, vlines = vlines,
                                row_sort=sort, slant=slant, cbar=True, cbar_label=cbar_label, units=units)
    
    # Write row statistics to CSV file
    row_stats.to_csv(csv_path+handle+'.csv')

    # Append cell tally row and add to Zip file 
    X.loc['cells'] = cell_tally
    X.to_csv(temp_csv)
    with zipfile.ZipFile(zip_file, zip_mode) as zipf:
        arc_name = handle+metric+'_subclass'+region.split()[0]+'_'+'.csv'
        zipf.write(temp_csv, arcname=arc_name)
    zip_mode = 'a'    
    print(arc_name,'added to:',zip_file,'\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)


In [ ]:
# Map all subclasses, write results to a PDF and a ZIP archive of CSVs
# Note: Good for Y21ss and Y21tenX, awkward for Y23 - too many subclasses! See above for neater Y23...

font_scale = 1.1
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})
metric = 'mean' # 'sum','mean', 'cell%', 'clus%'
row_stat='max' # 'max', 'mean', 'sum'

cluster_thresh_factor = 0.5
min_cluster = 10

region = 'All'

# Set minimum subclass size according to dataset choice
min_subclass = 10 if 'ss' in dataset else 100
#min_subclass = 10 if 'ss' in dataset else 1

min_subclass = 1 # Ad hoc

log = False if '%' in metric else True
threshold = 10 if 'ss' in dataset else 1
if metric == 'clus%': threshold *= cluster_thresh_factor * threshold
    
#threshold = 0.5 # Ad Hoc...

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root + 'allSubs_' + metric + '_' + row_stat + '_'
handle += 'minSubC' + str(min_subclass) + '_'
if 'clus' in metric: handle += 'minClus'+str(min_cluster)+'_'
if '%' in metric: handle += 'Thr'+str(threshold)+'_'
handle += sn
print(handle)

pdf_file = pdf_path+handle+'.pdf'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'

pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

# 
df = counts.loc[idx[types,regions,classes,subclasses], :]    

pre = len(df)
df = df.groupby('subclass').filter(lambda x: len(x) >= min_subclass) # filter out small subclasses
post = len(df)
print(pre - post, 'cells filtered out of', pre, 'due to subclass size <', min_subclass)
#if len(df) == 0: continue

if metric == 'sum': # Sum values
    X = df.groupby('subclass').sum().T

if metric == 'mean': # Find mean values
    X = df.groupby('subclass').mean().T

if metric == 'cell%': # Find percent expressing by cell
    X = 100 * (df >= threshold).astype(int).groupby('subclass').mean().T

if metric == 'clus%': # Find percent expressing by cluster
    pre = len(df)
    df = df.groupby('cluster').filter(lambda x: len(x) >= min_cluster) # filter out small clusters
    post = len(df)
    print(pre - post, 'cells filtered out of', pre, 'due due to cluster size <', min_cluster)
    cluster_mean = df.groupby(['subclass','cluster']).mean()
    cluster_hot = (cluster_mean >= threshold).astype(int)
    cluster_frac = cluster_hot.groupby('subclass').mean()
    X = 100 * cluster_frac.T

# Make two useful lists
column_labels = [subclass_shorts[s] for s in list(X.columns)]
column_colors = [subclass_colors[s] for s in list(X.columns)]

# Get unit tallies
cell_tally = df.groupby('subclass').count().iloc[:,0].T
cell_tally.index = column_labels

# Find zone boundaries if desired
if draw_vlines:
    vlines = []; zone = []
    for i, subclass in enumerate(X.columns):
        if subclass_classes[subclass] != zone: vlines += [i]; zone = subclass_classes[subclass]
    vlines += [len(X.columns)]
else: vlines = None

# Draw subclass heatmap
title = handle+'\nAll Regions ('+metric+')'
if '%' in metric: cbar_label = '%Counts>='+str(threshold); units =' %'
else: cbar_label = 'Mean_Counts'; units = ''

X.index = yticklabels
X.columns = column_labels
row_stats = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log,
                            row_stat=row_stat, log_floor_coef=log_floor_coef, vlines = vlines,
                            row_sort=sort, slant=slant, cbar=True, cbar_label=cbar_label, units=units)

# Write row statistics to CSV file
row_stats.to_csv(csv_path+handle+'.csv')

# Append cell tally row and add to Zip file 
X.loc['cells'] = cell_tally
X.to_csv(temp_csv)
with zipfile.ZipFile(zip_file, zip_mode) as zipf:
    arc_name = handle+metric+'_subclass'+region.split()[0]+'_'+'.csv'
    zipf.write(temp_csv, arcname=arc_name)
zip_mode = 'a'    
print(arc_name,'added to:',zip_file,'\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)

In [ ]:
# Map clusters by subclass, write results to a PDF and a ZIP archive of CSVs

font_scale = 1.1
sns.set_theme(font_scale=font_scale, rc={'patch.linewidth': 0})

metric = 'mean' # 'mean', 'cell%'

log = True if metric == 'mean' else False
threshold = 10 if 'ss' in dataset else 1
threshold *= cluster_thresh_factor * threshold

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root + 'clusters_' + metric + '_'
if '%' in metric: handle += 'T'+str(threshold)+'_'
handle += 'M'+str(min_cluster)+'_'
handle += sn
print(handle)

pdf_file = pdf_path+handle+'.pdf'
zip_file = zip_path+handle+'.zip'
temp_csv = temp_path+'temp.csv'

pdf = PdfPages(pdf_file)

idx = pd.IndexSlice
zip_mode = 'w'

#for region in [regions[0]]: # for quick short result
for subclass in subclasses:
    df = counts.copy()
    df = df.droplevel(['type','region','class'])
    df = df.loc[subclass]
            
    if metric == 'mean': # Find mean values
        X = df.groupby('cluster').mean().T
    
    if metric == 'cell%': # Find percent expressing by cell
        X = 100 * (df >= threshold).astype(int).groupby('cluster').mean().T

    # Make two useful lists
    column_labels = X.columns
    column_colors = [cluster_colors[s] for s in list(X.columns)]

    # Get unit tallies
    cell_tally = df.groupby('cluster').count().iloc[:,0].T
    cell_tally.index = column_labels
        

    # Draw subclass heatmap
    title = handle+'\nSubclass: '+subclass_shorts[subclass]+' ('+metric+')'
    if '%' in metric: cbar_label = '%Counts>='+str(threshold); units =' %'
    else: cbar_label = 'Mean_Counts'; units = ''

    X.index = yticklabels
    X.columns = column_labels
    f = pm.draw_heatmap(X, cell_tally, column_colors, title, pdf, row_norm=row_norm, log=log, 
                        log_floor_coef=log_floor_coef, vlines = None, row_sort=sort, slant=slant,
                        cbar=True, cbar_label=cbar_label, units=units) #, fontsize=fontsize)

    # Append cell tally row and add to Zip file 
    X.loc['cells'] = cell_tally
    X.to_csv(temp_csv)
    with zipfile.ZipFile(zip_file, zip_mode) as zipf:
        arc_name = handle+subclass+'.csv'
        zipf.write(temp_csv, arcname=arc_name)
    zip_mode = 'a'    
    print(arc_name,'added to:',zip_file,'\n\n')

#
pdf.close(); print('Graphic to PDF:', pdf_file)
